In [22]:
# pip install -U selenium

In [15]:
import requests
from bs4 import BeautifulSoup
import urllib
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [19]:
import time
from selenium import webdriver #allows you to programmatically interact with a browser on an operating system the way a real user would
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime
import smtplib #to send basic emails. Send emails with HTML content and attachments using the email package.
# Multipurpose Internet Mail Extensions (MIME) is an Internet standard that is used to support the transfer of single or multiple text and non-text attachments.
# Non-text attachments can include graphics, audio, and video files.
from email.mime.text import MIMEText
# helps in sending attachments via mail
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from bs4 import BeautifulSoup #a Python library that is used for web scraping purposes to pull the data out of HTML and XML files
url = 'https://www.ontario.ca/page/oinp-express-entry-notifications-interest'

def send_selenium_report():
#     dir_path = "/Users/490/Desktop/Web Scraping/"
#     files = ["fig1.png", "report.xlsx","Assigned Tickets.png"]

    # assign key email aspects to variables for easier future editing
    subject = "OINP Tech Draw has been picked"
    body = "This is an email with the desired report attached"
    sender_email = "riya.kashyap0198@gmail.com"
    receiver_email = ["riya.kashyap1598@gmail.com","vu3bsp1993@gmail.com"]
    rec = ", ".join(receiver_email) 
    password = "mkuwzwegqalynoww"
    
    # Create the email head (sender, receiver, and subject)
    email = MIMEMultipart()
    email["From"] = sender_email
    email["To"] = rec
    email["Subject"] = subject

#     body = MIMEText('The crs-cutoff score is {}. \n click the following url'.format(df['CRS score range'][0], url), 'html', 'utf-8')  
#     body = df

    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())
    html_doc = html
    soup = BeautifulSoup(html_doc, "lxml")
    tag = soup.head
    tag.string = 'The crs-cutoff score is {}. \n click the following url {}'.format(df['CRS score range'][0], url)

    part1 = MIMEText(soup, 'html')
    email.attach(part1)  # add message body (text or html)

#     for f in files:  # add files to the message
#         file_path = os.path.join(dir_path, f)
#         attachment = MIMEApplication(open(file_path, "rb").read(), _subtype="txt")
#         attachment.add_header('Content-Disposition','attachment', filename=f)
#         email.attach(attachment)

        
    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_email, password) #login with mail_id and password
    text = email.as_string()
    session.sendmail(sender_email, receiver_email, text)
    session.quit()


driver = webdriver.Chrome('C:/Users/chromedriver')
driver.get(url)
elem = driver.find_element_by_xpath("/321/*")
driver.find_element("xpath", "/321/*").click()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/321/*"))).send_keys("test")
time.sleep(3)
elem = driver.find_element_by_xpath('//div[@class="flex-table"]')
source_code = elem.get_attribute("outerHTML")
html_tables = pd.read_html(source_code)
df = html_tables[0]
date_list = df['Date/time NOIs issued'][0].split(" ")
month = date_list[0]
day = date_list[1][:-1]
year = date_list[2]
draw_date = datetime.datetime.strptime(month + day + year, '%B%d%Y')

current_day = datetime.datetime.now()
# current_day = datetime.datetime(2021,4,13)

if (current_day - draw_date).days == 0 and 'Targeted draw – Tech draw' in df['Notes'][0]:
    send_selenium_report()


C:\Users\User\AppData\Local\Temp\ipykernel_9104\273685401.py:69: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/chromedriver')


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [21]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
import smtplib
import ssl

smtp_server = "smtp.gmail.com"
port = 587  # For starttls

sender_email =  "riya.kashyap0198@gmail.com" # TODO: replace with your email address
receiver_email = ["riya.kashyap1598@gmail.com","vu3bsp1993@gmail.com"] # TODO: replace with your recipients
password = "mkuwzwegqalynoww" # TODO: replace with your 16-digit-character password 

# assuming these two values are from your analysis
score = 0.86
today_date = datetime.datetime.now()

# initialise message instance
msg = MIMEMultipart()
msg["Subject"] = "Training Job Notification on {}".format(today_date)
msg["From"] = sender_email
msg['To'] = ", ".join(receiver_email)

## Plain text
text = """\
This line is to demonstrate sending plain text."""

body_text = MIMEText(text, 'plain')  # 
msg.attach(body_text)  # attaching the text body into msg

html = """\
<html>
  <body>
    <p>Hi,<br>
    <br>
    This is to inform the training job has been completed. The AUC for the job on {} is {} <br>
    Thank you. <br>
    </p>
  </body>
</html>
"""

body_html = MIMEText(html.format(today_date, score), 'html')  # parse values into html text
msg.attach(body_html)  # attaching the text body into msg

## Image
img_name = 'Screenshot 2021-11-01 181846.png' # TODO: replace your image filepath/name
with open(img_name, 'rb') as fp:
    img = MIMEImage(fp.read())
    img.add_header('Content-Disposition', 'attachment', filename=img_name)
    msg.attach(img)

## Attachments in general
## Replace filename to your attachments. Tested and works for png, jpeg, txt, pptx, csv
filename = 'test.csv' # TODO: replace your attachment filepath/name
with open(filename, 'rb') as fp:
    attachment = MIMEApplication(fp.read())
    attachment.add_header('Content-Disposition', 'attachment', filename=filename)
    msg.attach(attachment)

context = ssl.create_default_context()
# Try to log in to server and send email
try:
    server = smtplib.SMTP(smtp_server, port)
    server.ehlo()  # check connection
    server.starttls(context=context)  # Secure the connection
    server.ehlo()  # check connection
    server.login(sender_email, password)

    # Send email here
    server.sendmail(sender_email, receiver_email, msg.as_string())

except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit()

FileNotFoundError: [Errno 2] No such file or directory: 'Screenshot 2021-11-01 181846.png'